In [1]:
import codecs
import lasio
import sys
import os
import re
import numpy as np
from tqdm import tqdm


In [2]:
def getFilePaths(path):
    list = os.listdir(path)
    list = [(path + '/' + list[i]).replace('\\', '/') for i in range(len(list))]
    paths = [fileOrPath for fileOrPath in list if os.path.isdir(fileOrPath)]
    files = [fileOrPath for fileOrPath in list if os.path.isfile(fileOrPath) and fileOrPath.lower().endswith('las')]
    for path in paths:
        files.extend(getFilePaths(path))
    return files
#(CHECK) Класс конвертации путей файлов в зависимости от системы
class PathConverter():
    def replaceSlash(self, filePath):
        platform = sys.platform
        slashMap = {'win32': '\\',
                    'cygwin': '\\',
                    'darwin': '/',
                    'linux2': '/'}
        if platform not in slashMap.keys(): platform = 'linux2'
        return filePath.replace('\\',slashMap[platform])
class WellClass:
    def __init__(self, name):
        self.name = name
        self.curves = {}
        self.keys = []
        self.list = None
        self.wellData = {}
        self.header = None
        self.dept = []
        self.curvesList = []
class Curve:
    dataX = []
    mnemonic = ''
    original_mnemonic = ''
    unit = ''
    descr = ''
    lasFile = ''
    def __init__(self, dX,  n):
        self.name = n
        self.dataX = dX
class LasInfo:
    def __init__(self, filePath, minDept, maxDept, step, encoding, las):
        self.filePath = filePath
        self.minDept = minDept
        self.maxDept = maxDept
        self.step = step
        self.encoding = encoding
        self.las = las
class CurveInfo:
    def __init__(self,unit,descr,original_mnemonic,mnemonic,file):
        self.unit = unit
        self.descr = descr
        self.original_mnemonic = original_mnemonic
        self.mnemonic = mnemonic
        self.file = file
#Доступные Python кодировки
encodincs = ['ascii', 'utf_8', 'cp1251', 'cp866', 'utf_8_sig', 'cp1252', 'big5',
             'big5hkscs', 'cp037', 'cp273', 'cp424', 'cp437',
             'cp500', 'cp720', 'cp737', 'cp775', 'cp850', 'cp852', 'cp855', 'cp856',
             'cp857', 'cp858', 'cp860', 'cp861', 'cp862', 'cp863', 'cp864', 'cp865',
             'cp869', 'cp874', 'cp875', 'cp932', 'cp949', 'cp950', 'cp1006',
             'cp1026', 'cp1125', 'cp1140', 'cp1250', 'cp1253',
             'cp1254', 'cp1255', 'cp1256', 'cp1257', 'cp1258', 'cp65001', 'euc_jp',
             'euc_jis_2004', 'euc_jisx0213', 'euc_kr', 'gb2312', 'gbk', 'gb18030',
             'hz', 'iso2022_jp', 'iso2022_jp_1', 'iso2022_jp_2', 'iso2022_jp_2004',
             'iso2022_jp_3', 'iso2022_jp_ext', 'iso2022_kr', 'latin_1', 'iso8859_2',
             'iso8859_3', 'iso8859_4', 'iso8859_5', 'iso8859_6', 'iso8859_7', 'iso8859_8',
             'iso8859_9', 'iso8859_10', 'iso8859_11', 'iso8859_13', 'iso8859_14',
             'iso8859_15', 'iso8859_16', 'johab', 'koi8_r', 'koi8_t', 'koi8_u', 'kz1048',
             'mac_cyrillic', 'mac_greek', 'mac_iceland', 'mac_latin2', 'mac_roman',
             'mac_turkish', 'ptcp154', 'shift_jis', 'shift_jis_2004', 'shift_jisx0213',
             'utf_32', 'utf_32_be', 'utf_32_le', 'utf_16', 'utf_16_be', 'utf_16_le', 'utf_7']
#Попытка поправить линию в шапке
def createLine(line):
    newLine = ''
    p = line.split(':')
    newLine = newLine + p[0] + '. :'
    for i in range(1,len(p)):
        newLine = newLine + p[i]
    return newLine
#Попытка локализовать и устранить ошибку в .las файле
def checkAndFixLas(file, encoding):
    f = codecs.open(file, 'r', encoding=encoding)
    text = f.read().split('\n')
    f.close()
    needFix = False
    i = 0
    while text[i].lower().find('~curve') == -1:
        i = i + 1
    i = i + 1
    while text[i].find('~') == -1:
        line = text[i]
        if line.find(':') != -1:
            if line.find('.') == -1 or line.find('.') > line.rfind(':'):
                needFix = True
                newLine = createLine(line)
                text[i] = newLine
        else:
            if line.find('.') != -1:
                needFix = True
                p = line.split('.')
                newLine = p[0] + '. :'
                for k in range(1,len(p)):
                    newLine = newLine + p[k]
                text[i] = newLine
        i = i + 1
    if needFix == True:
        print('Missed \'.\' in ~Curve information, trying to fix')
        file = file.replace('.las','').replace('.LAS','') + '(copy).las'
        f = codecs.open(file , 'w', encoding=encoding)
        for line in text:
            f.write(line + '\n')
        f.close()
        return file
    return file
# Интерполяция
def newMesh(CRV, DEPT, new_mesh):
    if abs(DEPT[0] - new_mesh[0]) < 1e-5 and len(new_mesh) == len(DEPT):
        return CRV
    else:
        new_crv = np.interp(new_mesh, DEPT, CRV, left=np.nan, right=np.nan)
        return new_crv

In [3]:
#Входная папка
path = "C:/Job/Python/Диплом/шегурч"





print('Starting read...')
regex = r"[^а-яА-Яa-zA-Z\s\0\^\$\`\~\!\@\"\#\№\;\%\^\:\?\*\(\)\-\_\=\+\\\|\[\]\{\}\,\.\/\'\d]"

#Отсев сважин по имени, ignoreLiterals будет игнорировать все, кроме цифр в имени скважины, если True
ignoreLiterals  = True
regWellName = r"[^0-9]"

#Функция сбора лас файлов в папке и во всех вложенных папках

converter = PathConverter()
files = getFilePaths(os.path.abspath(converter.replaceSlash(path)))
wellsLasFiles = {}
wells = []
log = []
smokerSFiles = []
wellsInfo = {}
importLog = {}
maxVal = len(files)
print('Files to read : ',maxVal)

Starting read...
Files to read :  12


In [4]:
for file in tqdm(files):
    file = converter.replaceSlash(file)
    #print(file)
    enc = 'None'
    maxLen = 999999
    maybeEnc = 'ascii'
    for e in encodincs:
        try:
            cod = codecs.open(file, 'r', encoding=e)
            l = str(cod.read())
            l = l.replace(' ','')
            check = re.findall(regex,l)
            curLen = len(check)
            if curLen != 0:
                if curLen < maxLen:
                    maxLen = curLen
                    maybeEnc = e
            else:
                enc = e
                break
        except:
            pass
    if enc == 'None':
        #print('Warning: trying to set best encoding : ' + maybeEnc)
        enc = maybeEnc
    if enc != 'None':
        try:
            try:
                f = lasio.read(file, encoding=enc)
            except:
                newFile = checkAndFixLas(file,enc)
                f = lasio.read(newFile,encoding=enc,ignore_header_errors=True)
            if str(f.well['WELL'].value) == '':
                log.append(file)
                log.append('No wellname')
                #print('No wellname')
                smokerSFiles.append(file)
                continue
            try:
                lasInfoObj = LasInfo(file, f[f.keys()[0]][0], f[f.keys()[0]][-1],
                        f[f.keys()[0]][1] - f[f.keys()[0]][0], enc, f)
            except Exception as exc:
                log.append(file)
                log.append(str(exc))
                #print(str(exc))
                continue
            wellName = re.sub(regWellName,'',str(f.well['WELL'].value)).replace(' ','') if ignoreLiterals == True else str(f.well['WELL'].value)
            if not wellName in wellsLasFiles.keys():
                wellsLasFiles[wellName] = []
            wellsLasFiles[wellName].append(lasInfoObj)
        except Exception as e:
            log.append(file)
            log.append(str(e))
            smokerSFiles.append(file)
            #print('Error while read')
    else:
        pass
        #print('Encoding not found')
print('Wells to handle: ',len(wellsLasFiles))
print('Bad files:',len(smokerSFiles))

100%|██████████| 12/12 [00:16<00:00,  1.27s/it]


Wells to handle:  12
Bad files: 0


In [5]:
for wellName in wellsLasFiles.keys():
            try:
                globalMinDept = (wellsLasFiles.get(wellName)[0]).minDept
                globalMaxDept = (wellsLasFiles.get(wellName)[0]).maxDept
                globalMinStep = (wellsLasFiles.get(wellName)[0]).step
                print('LAS FILES COUNT FOR ' + str(wellName) + ' : ' + str(len(wellsLasFiles.get(wellName))))
                for lasInfoObject in wellsLasFiles.get(wellName):
                    min = lasInfoObject.minDept
                    max = lasInfoObject.maxDept
                    step = lasInfoObject.step
                    if min < globalMinDept:
                        globalMinDept = min
                    if max > globalMaxDept:
                        globalMaxDept = max
                globalMinStep = 0.1
                newWell = WellClass(wellName)
                dept = np.round(np.arange(globalMinDept, globalMaxDept, globalMinStep), 1)
                if dept[-1] != np.round(globalMaxDept,1):
                    dept = np.hstack((dept, np.round(globalMaxDept,1)))
                globalShape = len(dept)
                curves = {}
                curvesInfo = {}
                importLog[wellName] = []
                curves['DEPT'] = dept
            except Exception as exc:
                print(exc)
            unexpectedError = False
            downloadedFiles = 0
            for lasInfoObject in wellsLasFiles.get(wellName):
                try:
                    crvNames = [lasInfoObject.las.keys()[i] for i in range(0,len(lasInfoObject.las.keys()))]
                    if 'UNKNOWN' in crvNames:
                        log.append(lasInfoObject.filePath)
                        log.append('Unexpected curve name')
                        print('Unknown curve in ' + lasInfoObject.filePath)
                        smokerSFiles.append(lasInfoObject.filePath)
                        continue
                    crvs = ''
                    for crv in range(1,len(lasInfoObject.las.curves.keys())-1):
                        crvs = crvs + lasInfoObject.las.curves.keys()[crv] + ', '
                    crvs = crvs + lasInfoObject.las.curves.keys()[len(lasInfoObject.las.curves.keys())-1]
                    importLog[wellName].append(lasInfoObject.filePath+' |/+/| ' + crvs)
                    previousInfo = lasInfoObject.las.curves
                    localDept = np.round(lasInfoObject.las[lasInfoObject.las.keys()[0]], 1)
                    try:
                        curvesInfo['DEPT'] = CurveInfo(previousInfo[0].unit,previousInfo[0].descr,'DEPT','DEPT',lasInfoObject.filePath)
                    except:
                        curvesInfo['DEPT'] = CurveInfo('','','DEPT','DEPT',lasInfoObject.filePath)
                    for i in range(1, len(lasInfoObject.las.keys())):
                        try:
                            try:
                                curveName = lasInfoObject.las.keys()[i]
                                newCurveName = curveName
                            except:
                                print('step1')
                                continue
                            try:
                                if curveName in curves.keys():
                                    count = 1
                                    while newCurveName in curves.keys():
                                        if count > 1:
                                            newCurveName = newCurveName[:-1]
                                        newCurveName = newCurveName + str(count)
                                        count = count + 1
                            except:
                                pass
                            newCurve = newMesh(lasInfoObject.las[curveName], localDept, dept)
                            curves[newCurveName] = newCurve
                            try:
                                curvesInfo[newCurveName] = CurveInfo(previousInfo[curveName].unit,
                                                                     previousInfo[curveName].descr,
                                                                     previousInfo[curveName].original_mnemonic,
                                                                     curveName,
                                                                     lasInfoObject.filePath)
                            except:
                                pass
                                curvesInfo[newCurveName] = CurveInfo('','',curveName,curveName,lasInfoObject.filePath)
                        except Exception as exc:
                            print(exc)
                    downloadedFiles += 1
                except Exception as e:
                    log.append(lasInfoObject.filePath)
                    log.append(str(e))
                    smokerSFiles.append(lasInfoObject.filePath)
                    print('Union error')
            if downloadedFiles == 0:
                unexpectedError = True
            try:
                if not unexpectedError:
                    wellsInfo[wellName] = curvesInfo
                    newWell.curves = curves
                    newWell.keys = curves.keys()
                    wells.append(newWell)
            except:
                print('Unexpected error')
                pass
print('Complete wells: ',len(wells))
#print([well.name for well in wells])
print('Bad files :',len(smokerSFiles))

LAS FILES COUNT FOR 16400 : 1
LAS FILES COUNT FOR 16493 : 1
LAS FILES COUNT FOR 16496 : 1
LAS FILES COUNT FOR 4733 : 1
LAS FILES COUNT FOR 4734 : 1
LAS FILES COUNT FOR 4735 : 1
LAS FILES COUNT FOR 4736 : 1
LAS FILES COUNT FOR 4737 : 1
LAS FILES COUNT FOR 4738 : 1
LAS FILES COUNT FOR 4739 : 1
LAS FILES COUNT FOR 4740 : 1
LAS FILES COUNT FOR 4741 : 1
Complete wells:  12
Bad files : 0


In [6]:
wellsToMerge = {}
for well in wells:
    wellsToMerge[well.name] = well
    for curveName in well.curves.keys():
        well.curvesList.append(Curve(well.curves[curveName],curveName))
print(wellsToMerge.keys())

dict_keys(['16400', '16493', '16496', '4733', '4734', '4735', '4736', '4737', '4738', '4739', '4740', '4741'])


In [7]:

class IntersectionInformation():
    def __init__(self,k,corr,length,intersection,isNone,curveUpper,curveDown):
        self.k = k
        self.corr = corr
        self.intersection = intersection
        self.length = length
        self.tested = False
        self.isNone = isNone
        self.curveUpper = curveUpper
        self.curveDown = curveDown

In [16]:
#Класс сшивающий кривые с пересечением
class CurvesMerger():
    def __init__(self,wells):
        self.wells = wells
        self.mergeLog = []
        self.haveIntersectionOnceFlag = False
    #Посчитать пересечение и его длину
    def takeIntersection(self,curve1,curve2,dept):
        intersection = None
        length = None
        dept1 = dept[np.isfinite(curve1.dataX)]
        dept2 = dept[np.isfinite(curve2.dataX)]
        dept3 = dept[np.logical_and(np.isfinite(curve1.dataX),np.isfinite(curve2.dataX))]
        if len(dept3) == 0 or (dept2[-1] <= dept1[-1]):
            return intersection,length
        else:
            intersection = [dept3[0],dept3[-1]]
            length = intersection[1]-intersection[0]
            return intersection,length

    #Посчитать корреляцию и отношение
    def takeCorrelationAndCoef(self,curve1,curve2,dept,intersection):
        ind = np.logical_and(np.logical_and(dept >= intersection[0],dept <= intersection[1]),np.logical_and(np.isfinite(curve1.dataX),np.isfinite(curve2.dataX)))
        crv1 = curve1.dataX[ind]
        crv2 = curve2.dataX[ind]
        corr = np.corrcoef(crv1,crv2)
        mmm = crv2/crv1
        mmm1 = crv1/crv2
        m1 = np.nanmean(crv1)
        m2 = np.nanmean(crv2)
        mmm[np.where(mmm == np.inf)] = 0
        mmm1[np.where(mmm1 == np.inf)] = 0
        k1 = np.nanmean(mmm)
        k = k1
        k2 = np.nanmean(mmm1)
        k3 = m1/m2
        k4 = m2/m1
        if np.abs(1-k2)<np.abs(1-k):
            k = k2
        if np.abs(1-k3)<np.abs(1-k):
            k = k3
        if np.abs(1-k4)<np.abs(1-k):
            k = k4
        return corr[1][0],k

    #Посчитать матрицу всех значений
    def takeMatrix(self,wellName, curvesToMerge):
        dept = self.wells[wellName].curves['DEPT']
        a = []
        #Создать матрицу
        for i in range(0,len(curvesToMerge)):
            mas = []
            for k in range(0,i+1):
                mas.append(IntersectionInformation(None,None,None,None,True,None,None))
            for j in range(i + 1, len(curvesToMerge)):
                try:
                    curveUpper = None
                    curveDown = None
                    dept1 = dept[np.isfinite(curvesToMerge[i].dataX)]
                    dept2 = dept[np.isfinite(curvesToMerge[j].dataX)]
                    if len(dept1) != 0 and len(dept2) != 0:
                        if dept1[0] <= dept2[0]:
                            if dept1[0] == dept2[0]:
                                if dept1[-1] >= dept2[-1]:
                                    curveUpper = curvesToMerge[i]
                                    curveDown = curvesToMerge[j]
                                else:
                                    curveUpper = curvesToMerge[j]
                                    curveDown = curvesToMerge[i]
                            else:
                                curveUpper = curvesToMerge[i]
                                curveDown = curvesToMerge[j]
                        else:
                            curveUpper = curvesToMerge[j]
                            curveDown = curvesToMerge[i]
                        intersection, length = self.takeIntersection(curveUpper, curveDown,dept)
                        corr = None
                        k = None
                        if length != None:
                            corr, k = self.takeCorrelationAndCoef(curveUpper, curveDown, dept, intersection)
                            isNone = False
                        else:
                            isNone = True
                        mas.append(IntersectionInformation(k, corr, length, intersection, isNone, curveUpper, curveDown))
                    else:
                        mas.append(IntersectionInformation(None, None, None, None, True, None, None))
                except Exception as e:
                    print(e)
                    print('Fail in takeMatrix, well - ', wellName)
            if len(mas) > 0:
                a.append(mas)
        return a

    #Найти в матрице куски меньше 2м с коэф. [0.4 , 1.6]
    def takeLittleParts(self,a):
        mergeIndexes = []
        littleK = []
        for i in range(0, len(a)):
            for j in range(0, len(a[i])):
                if a[i][j].isNone != True:
                    ind = [i, j]
                    if a[i][j].length < 2 and a[i][j].k <= 1.6 and a[i][j].k >= 0.4 and a[i][j].tested == False:
                        mergeIndexes.append(ind)
                        littleK.append(a[i][j])
        return mergeIndexes, littleK

    #Найти в матрице среди положительной корреляции, наиболее близкий к 1 коэф.
    def takeBestK(self,a,trueName):
        bestK = None
        indexes = []
        for i in range(0, len(a)):
            for j in range(0, len(a[i])):
                if a[i][j].isNone == False:
                    if (a[i][j].corr > 0 and a[i][j].k <= 1.6 and a[i][j].k >= 0.4 and a[i][j].tested == False) or (trueName.upper().startswith('PS') == True):
                        if bestK == None:
                            bestK = a[i][j]
                            indexes = [i, j]
                        else:
                            if np.abs(1-a[i][j].k) < np.abs(1-bestK.k):
                                bestK = a[i][j]
                                indexes = [i, j]
        return indexes,bestK
    
    #Проверить есть ли пересечение с другими кривыми хоть 1 раз
    def haveIntersectionOnce(self,checkCurve,curvesToMerge,wellName):
        dept = self.wells[wellName].curves['DEPT']
        haveIntersection = False
        for curve in curvesToMerge:
            if curve != checkCurve:
                curveUpper = None
                curveDown = None
                dept1 = dept[np.isfinite(checkCurve.dataX)]
                dept2 = dept[np.isfinite(curve.dataX)]
                if len(dept1) != 0 and len(dept2) != 0:
                    if dept1[0] <= dept2[0]:
                        curveUpper = checkCurve
                        curveDown = curve
                    else:
                        curveUpper = curve
                        curveDown = checkCurve
                    intersection, length = self.takeIntersection(curveUpper, curveDown, dept)
                else:
                    intersection = None
                    length = None
                if length != None:
                    haveIntersection = True
                    break
        return haveIntersection


    #основная часть тут
    def mergeCurves(self,wellName, curvesToMerge, trueName):
        dept = self.wells[wellName].curves['DEPT']
        self.curvesToDelete = []
        curvesToExit = []
        mergeCount = 0
        a = self.takeMatrix(wellName,curvesToMerge)
        ###убрать куски <2 м
        mergeIndexes, littleK = self.takeLittleParts(a)
        while len(mergeIndexes) > 0:

            newCurve,appendLog = self.mergePls(littleK[0],
                                     trueName,
                                     wellName)
            if len(newCurve) == 1:
                newCurve = newCurve[0]
                c = littleK[0].curveUpper
                self.curvesToDelete.append(curvesToMerge.pop(curvesToMerge.index(littleK[0].curveUpper)))
                self.curvesToDelete.append(curvesToMerge.pop(curvesToMerge.index(littleK[0].curveDown)))
                if mergeCount > 0:
                    newCurve.name = trueName + str(mergeCount)
                    newCurve.original_mnemonic = trueName
                else:
                    newCurve.name = trueName
                    newCurve.original_mnemonic = trueName
                appendLog[-1] = "Сшиты в " + newCurve.name
                self.mergeLog.append(appendLog)
                mergeCount = mergeCount + 1
                newCurve.unit = c.unit
                newCurve.descr = c.descr
                #newCurve.original_mnemonic = c.original_mnemonic
                curvesToMerge.append(newCurve)
            else:
                appendLog[-1] = "Не сшиты"
                self.mergeLog.append(appendLog)
                littleK[0].tested = True
            a = self.takeMatrix(wellName, curvesToMerge)
            mergeIndexes, littleK = self.takeLittleParts(a)
        a = self.takeMatrix(wellName, curvesToMerge)
        ###TODO удалить оставшиеся куски <2 м
        i = 0
        while i < len(curvesToMerge):
            haveGoodPart = self.haveIntersectionOnce(curvesToMerge[i],curvesToMerge,wellName)
            if haveGoodPart == False:
                c = curvesToMerge.pop(i)
                curvesToExit.append(c)
                a.pop(i)
            else:
                i = i + 1
        a = self.takeMatrix(wellName,curvesToMerge)
        #Основной цикл
        indexes, bestK = self.takeBestK(a,trueName)
        while bestK != None:
            newCurve, appendLog = self.mergePls(bestK,trueName,wellName)
            if len(newCurve) == 1:
                newCurve = newCurve[0]
                c = bestK.curveUpper
                self.curvesToDelete.append(curvesToMerge.pop(curvesToMerge.index(bestK.curveUpper)))
                self.curvesToDelete.append(curvesToMerge.pop(curvesToMerge.index(bestK.curveDown)))
                if mergeCount > 0:
                    newCurve.name = trueName + str(mergeCount)
                    newCurve.original_mnemonic = trueName
                else:
                    newCurve.name = trueName
                    newCurve.original_mnemonic = trueName
                appendLog[-1] = "Сшиты в " + newCurve.name
                self.mergeLog.append(appendLog)
                mergeCount = mergeCount + 1
                newCurve.unit = c.unit
                newCurve.descr = c.descr
                curvesToMerge.append(newCurve)
                a = self.takeMatrix(wellName, curvesToMerge)
            else:
                appendLog[-1] = "не сшиты"
                self.mergeLog.append(appendLog)
                a[indexes[0]][indexes[1]].tested = True
            indexes, bestK = self.takeBestK(a,trueName)
        for i in range(0,len(a)):
            for j in range(i+1,len(a[i])):
                #print(a[i][j].curveUpper.name,a[i][j].curveDown.name,a[i][j].length,a[i][j].intersection,a[i][j].corr,a[i][j].k,a[i][j].isNone)
                self.mergeLog.append([wellName,a[i][j].curveUpper.name,a[i][j].curveDown.name,
                                      str(a[i][j].length),str(a[i][j].corr),str(a[i][j].k),
                                      "","","",""])
        for curve in curvesToMerge:
            curvesToExit.append(curve)
        return curvesToExit
    
    #Сшивка двух кривых
    def mergePls(self,intersectionInformation,trueName,wellName):
        appendLog = []
        dept = self.wells[wellName].curves['DEPT']
        newCurves = []
        newCurve = Curve(np.ones(len(dept)) * np.nan, len(dept))
        curve1 = intersectionInformation.curveUpper
        curve2 = intersectionInformation.curveDown
        curve1Dept = dept[np.isfinite(curve1.dataX)]
        curve2Dept = dept[np.isfinite(curve2.dataX)]
        if trueName == 'PS':
            ind = np.logical_and(dept >= intersectionInformation.intersection[0],
                                 dept <= intersectionInformation.intersection[1])
            part1 = curve1.dataX[ind]
            part2 = curve2.dataX[ind]

            a1 = np.nanmean(part1)
            a2 = np.nanmean(part2)
            partBefore = np.logical_and(dept >= curve1Dept[0], dept < intersectionInformation.intersection[1])
            partAfter = np.logical_and(dept >= intersectionInformation.intersection[1], dept <= curve2Dept[-1])
            newCurve.dataX[partBefore] = curve1.dataX[partBefore]
            newCurve.dataX[partAfter] = curve2.dataX[partAfter] - (a2 - a1)
            appendLog = [
                                wellName,
                                curve1.name,
                                curve2.name,
                                str(intersectionInformation.length),
                                str(intersectionInformation.corr),
                                str(intersectionInformation.k),
                                str(intersectionInformation.intersection[1]),
                                "К "+curve2.name+ " прибавлено ("+str(-(a2-a1))+")",
                                ""]
            newCurves.append(newCurve)
            return newCurves, appendLog
        l = intersectionInformation.length
        if l < 2:
            ind = np.logical_and(dept >= intersectionInformation.intersection[0],
                                 dept <= intersectionInformation.intersection[1])
            part1 = curve1.dataX[ind]
            part2 = curve2.dataX[ind]
            a = np.nanmean(part1 / part2)
            partBefore = np.logical_and(dept >= curve1Dept[0], dept < intersectionInformation.intersection[0])
            partAfter = np.logical_and(dept > intersectionInformation.intersection[1], dept <= curve2Dept[-1])
            newCurve.dataX[partBefore] = curve1.dataX[partBefore]
            newCurve.dataX[ind] = curve2.dataX[ind] * a
            newCurve.dataX[partAfter] = curve2.dataX[partAfter]
            appendLog = [
                wellName,
                curve1.name,
                curve2.name,
                str(intersectionInformation.length),
                str(intersectionInformation.corr),
                str(intersectionInformation.k),
                str(intersectionInformation.intersection[0]),
                "Объединить кривые осреднив интервал, коэф. = "+str(a),
                ""]
            newCurves.append(newCurve)
            return newCurves, appendLog
        else:
            k = intersectionInformation.k
            if k <= 1.1 and k >= 0.9:
                ind = np.logical_and(dept >= intersectionInformation.intersection[0],
                                     dept <= intersectionInformation.intersection[1])
                part1 = curve1.dataX[ind]
                part2 = curve2.dataX[ind]
                a = np.nanmean(part1 / part2)
                partBefore = np.logical_and(dept >= curve1Dept[0], dept < intersectionInformation.intersection[1])
                partAfter = np.logical_and(dept >= intersectionInformation.intersection[1], dept <= curve2Dept[-1])
                newCurve.dataX[partBefore] = curve1.dataX[partBefore]
                newCurve.dataX[partAfter] = curve2.dataX[partAfter]
                appendLog = [
                    wellName,
                    curve1.name,
                    curve2.name,
                    str(intersectionInformation.length),
                    str(intersectionInformation.corr),
                    str(intersectionInformation.k),
                    str(intersectionInformation.intersection[1]),
                    "Объединить кривые",
                    ""]
                newCurves.append(newCurve)
                return newCurves, appendLog
            elif k <= 1.6 and k >= 0.4:
                ind = np.logical_and(dept >= intersectionInformation.intersection[0],
                                     dept <= intersectionInformation.intersection[1])
                deptPart = [intersectionInformation.intersection[0], intersectionInformation.intersection[1]]
                parts = []
                # a + i/n*(b-a)
                n, m = divmod((intersectionInformation.intersection[1] - intersectionInformation.intersection[0]), 2)
                for i in range(0, int(n)):
                    # parts.append([np.round(bestTopBot[1]+i/int(n)*(bestTopBot[1]-topBot[0]),2),
                    #              np.round(bestTopBot[1]+(i+1)/int(n)*(bestTopBot[1]-topBot[0]),2)])
                    parts.append([intersectionInformation.intersection[0] + i * 2, intersectionInformation.intersection[0] + (i + 1) * 2])
                if parts[-1][1] != intersectionInformation.intersection[1]:
                    parts.append([parts[-1][1], intersectionInformation.intersection[1]])
                usefulPart = None
                for part in parts:
                    partInd = np.logical_and(dept >= part[0], dept <= part[1])
                    curK = np.nanmean(curve2.dataX[partInd] / curve1.dataX[partInd])
                    if curK >= 0.9 and curK <= 1.1:
                        usefulPart = part
                if usefulPart != None:
                    #self.mergeLog.append('Найдено новое начало для сшивки на глубине ' + str(usefulPart[1]))
                    #TODO Доделать сшивку тут
                    needTopBot = [usefulPart[1],curve2Dept[-1]]
                    cutInd = np.logical_and(dept >= needTopBot[0], dept <= needTopBot[1])
                    partBefore = np.logical_and(dept >= curve1Dept[0], dept < usefulPart[1])
                    newCurve.dataX[partBefore] = curve1.dataX[partBefore]
                    newCurve.dataX[cutInd] = curve2.dataX[cutInd]
                    #Должен возвращать массив кривых
                    appendLog = [
                        wellName,
                        curve1.name,
                        curve2.name,
                        str(intersectionInformation.length),
                        str(intersectionInformation.corr),
                        str(intersectionInformation.k),
                        str(needTopBot[0]),
                        "Объединить кривые",
                        ""]
                    newCurves.append(newCurve)
                    return newCurves, appendLog
                else:
                    appendLog = [
                        wellName,
                        curve1.name,
                        curve2.name,
                        str(intersectionInformation.length),
                        str(intersectionInformation.corr),
                        str(intersectionInformation.k),
                        "нет",
                        "Оставить обе кривые",
                        ""]
                    newCurves.append(curve1)
                    newCurves.append(curve2)
                    #self.mergeLog.append('Не найдено')
                    return newCurves, appendLog
    
    #Запуск функции сшивки
    def mergeAllCurves(self,checkedWells = None,curveGroups = None,checkRegistry = None):
        print('Merge start')
        if checkedWells == None or curveGroups == None or checkRegistry == None:
            if len(self.wells) > 0:
                for wellName in self.wells.keys():
                    self.haveIntersectionOnceFlag = False
                    print(wellName)
                    curvesToMerge = {'GK':[],'NGK':[],'PS':[],'DS':[],'PZ':[],'BK':[],'IK':[],'MPZ':[],
                                     'GZ1':[],'GZ2':[],'GZ3':[],'GZ4':[],'GZ5':[],'GZ6':[],'REZ':[],
                                     'GGKP':[],'U1':[],'U2':[],'U3':[],'DT':[],'MGZ':[], 'GZK' : [], 'MBK' : [], 'KP' : []}
                    for curve in self.wells[wellName].curvesList:
                        for name in curvesToMerge.keys():
                            if curve.name.upper().startswith(name) == True:
                                curvesToMerge[name].append(curve)
                                break
                    for trueName in curvesToMerge.keys():
                        if len(curvesToMerge[trueName]) > 1:
                            self.curvesToMerge = curvesToMerge[trueName]
                            print('Curve '+trueName+' started')
                            newCurves = self.mergeCurves(wellName, curvesToMerge[trueName], trueName)
                            for curve in self.curvesToDelete:
                                try:
                                    self.wells[wellName].curvesList.remove(curve)
                                except:
                                    pass
                            i = 0
                            while i < len(newCurves):
                                curve = newCurves[i]
                                if curve not in self.wells[wellName].curvesList:
                                    self.wells[wellName].curvesList.append(curve)
                                i = i + 1
                            print('Curve done')
        else:
            for wellName in checkedWells:
                print(wellName)
                curvesToMerge = {}
                for name in curveGroups.keys():
                    curvesToMerge[name] = []
                    for curve in self.wells[wellName].curvesList:
                        for key in curveGroups[name]:
                            if key.find('*') == -1 or key.find('*') != len(key) -1:
                                if checkRegistry == True:
                                    if curve.original_mnemonic == key:
                                        curvesToMerge[name].append(curve)
                                        break
                                else:
                                    if curve.original_mnemonic.upper() == key.upper():
                                        curvesToMerge[name].append(curve)
                                        break
                            elif key.find('*') == len(key) -1:
                                if checkRegistry == True:
                                    if len(re.findall(r"\b"+key.replace('*','+'),curve.original_mnemonic)) != 0:
                                        curvesToMerge[name].append(curve)
                                        break
                                else:
                                    if len(re.findall(r"\b"+key.upper().replace('*','+'),curve.original_mnemonic.upper())) != 0:
                                        curvesToMerge[name].append(curve)
                                        break
                                    
                for trueName in curvesToMerge.keys():
                    if len(curvesToMerge[trueName]) > 1:
                        self.curvesToMerge = curvesToMerge[trueName]
                        print('Curve ' + trueName + ' started')
                        newCurves = self.mergeCurves(wellName, curvesToMerge[trueName], trueName)
                        for curve in self.curvesToDelete:
                            try:
                                self.wells[wellName].curvesList.remove(curve)
                            except:
                                pass
                        i = 0
                        while i < len(newCurves):
                            curve = newCurves[i]
                            if curve not in self.wells[wellName].curvesList:
                                self.wells[wellName].curvesList.append(curve)
                            i = i + 1
                        print('Curve done')
        print('Merge done')
    #Запуск сшивки кривых без пересечения
    def mergeNoIntersection(self,checkedWells = None,curveGroups = None,checkRegistry = None):
        print('Union start')
        if checkedWells == None or curveGroups == None or checkRegistry == None:
            if len(self.wells) > 0:
                for wellName in self.wells.keys():
                    self.haveIntersectionOnceFlag = False
                    print(wellName)
                    curvesToMerge = {'GK': [], 'NGK': [], 'PS': [], 'DS': [], 'PZ': [], 'BK': [], 'IK': [], 'MPZ': [],
                                     'GZ1': [], 'GZ2': [], 'GZ3': [], 'GZ4': [], 'GZ5': [], 'GZ6': [], 'REZ': [],
                                     'GGKP': [], 'U1': [], 'U2': [], 'U3': [], 'DT': [], 'MGZ': [], 'GZK' : [], 'MBK' : [], 'KP' : []}
                    for curve in self.wells[wellName].curvesList:
                        for name in curvesToMerge.keys():
                            if curve.name.upper().startswith(name) == True:
                                curvesToMerge[name].append(curve)
                                break
                    for trueName in curvesToMerge.keys():
                        if len(curvesToMerge[trueName]) > 1:
                            self.curvesToMerge = curvesToMerge[trueName]
                            print('Curve ' + trueName + ' started')
                            newCurves = self.mergeNoIntersectionCurves(wellName, curvesToMerge[trueName], trueName)
                            for curve in self.curvesToDelete:
                                try:
                                    self.wells[wellName].curvesList.remove(curve)
                                except:
                                    pass
                            i = 0
                            while i < len(newCurves):
                                curve = newCurves[i]
                                if curve not in self.wells[wellName].curvesList:
                                    self.wells[wellName].curvesList.append(curve)
                                i = i + 1
                            print('Curve done')
        else:
            for wellName in checkedWells:
                print(wellName)
                curvesToMerge = {}
                for name in curveGroups.keys():
                    curvesToMerge[name] = []
                    for curve in self.wells[wellName].curvesList:
                        for key in curveGroups[name]:
                            if key.find('*') == -1 or key.find('*') != len(key) - 1:
                                if checkRegistry == True:
                                    if curve.original_mnemonic == key:
                                        curvesToMerge[name].append(curve)
                                        break
                                else:
                                    if curve.original_mnemonic.upper() == key.upper():
                                        curvesToMerge[name].append(curve)
                                        break
                            elif key.find('*') == len(key) - 1:
                                if checkRegistry == True:
                                    if len(re.findall(r"\b" + key.replace('*', '+'), curve.original_mnemonic)) != 0:
                                        curvesToMerge[name].append(curve)
                                        break
                                else:
                                    if len(re.findall(r"\b" + key.upper().replace('*', '+'),
                                                      curve.original_mnemonic.upper())) != 0:
                                        curvesToMerge[name].append(curve)
                                        break
                for trueName in curvesToMerge.keys():
                    if len(curvesToMerge[trueName]) > 1:
                        self.curvesToMerge = curvesToMerge[trueName]
                        print('Curve ' + trueName + ' started')
                        newCurves = self.mergeNoIntersectionCurves(wellName, curvesToMerge[trueName], trueName)
                        for curve in self.curvesToDelete:
                            try:
                                self.wells[wellName].curvesList.remove(curve)
                            except:
                                pass
                        i = 0
                        while i < len(newCurves):
                            curve = newCurves[i]
                            if curve not in self.wells[wellName].curvesList:
                                self.wells[wellName].curvesList.append(curve)
                            i = i + 1
                        print('Curve done')
        print('Union end')
        
    #Проверка на хотя бы одно пересечение
    def anyIntersections(self,checkCurve, curvesToMerge, wellName):
        dept = self.wells[wellName].curves['DEPT']
        for curve in curvesToMerge:
            if curve != checkCurve:
                check = dept[np.logical_and(np.isfinite(checkCurve.dataX), np.isfinite(curve.dataX))]
                if len(check) != 0:
                    return True
        return False

    #Основная часть сшивки без пересечения
    def mergeNoIntersectionCurves(self,wellName, curvesToMerge, trueName):
        curvesToExit = []
        self.curvesToDelete = []
        mergeCount = 0
        i = 0
        while i < len(curvesToMerge):
            curvei = curvesToMerge[i]
            curveiMerged = False
            haveIntersection1 = self.anyIntersections(curvei, curvesToMerge, wellName)
            if haveIntersection1 == False:
                j = i + 1
                while j < len(curvesToMerge):
                    curvej = curvesToMerge[j]
                    curvejMerged = False
                    haveIntersection2 = self.anyIntersections(curvej,curvesToMerge,wellName)
                    if haveIntersection2 == False:
                        mergeCount += 1
                        curveiMerged = True
                        curvejMerged = True
                        newCurve = self.mergeWithoutIntersection(curvei,curvej,wellName)
                        c = curvesToMerge.pop(curvesToMerge.index(curvei))
                        self.curvesToDelete.append(curvesToMerge.pop(curvesToMerge.index(curvej)))
                        self.curvesToDelete.append(c)

                        self.mergeLog.append([wellName,curvei.name,curvej.name,"Сшиты без пересечения"])
                        newCurve.name = trueName + '__' + str(mergeCount)
                        newCurve.descr = c.descr
                        newCurve.unit = c.unit
                        newCurve.original_mnemnic = trueName
                        curvesToMerge.append(newCurve)
                        i = 0
                        break
                    else:
                        j += 1
            else:
                i += 1
            if curveiMerged == False:
                i += 1
        for curve in curvesToMerge:
            curvesToExit.append(curve)
        return curvesToExit
    
    #Сшивка кривых
    def mergeWithoutIntersection(self,curve1,curve2,wellName):
        dept = self.wells[wellName].curves['DEPT']
        newCurve = Curve(np.ones(len(dept))*np.nan,len(dept))
        ind1 = np.isfinite(curve1.dataX)
        newCurve.dataX[ind1] = curve1.dataX[ind1]
        ind1 = np.isfinite(curve2.dataX)
        newCurve.dataX[ind1] = curve2.dataX[ind1]
        return newCurve

In [17]:
merger = CurvesMerger(wellsToMerge)

In [18]:
for wellName in wellsToMerge.keys():
    print(wellName)
    print([curve.name for curve in wellsToMerge[wellName].curvesList])


16400
['DEPT', 'BK', 'BK_1', 'DS', 'GK', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZK', 'MGZ', 'MPZ', 'NGK', 'REZ', 'PS', 'PZ', 'IK']
16493
['DEPT', 'BK', 'DS', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZK', 'MGZ', 'MPZ', 'PS', 'U1', 'U2', 'U3', 'GK', 'NGK', 'PZ', 'IK', 'REZ']
16496
['DEPT', 'BK', 'DS', 'GZ1', 'GZ5', 'GZ5_2', 'GZ5_1', 'IK', 'MGZ', 'MPZ', 'REZ', 'REZ_1', 'U3_1', 'GK', 'NGK', 'PS', 'PZ', 'U1', 'U2', 'U3', 'GZ2', 'GZ3', 'GZ4', 'GZK']
4733
['DEPT', 'DS', 'IK', 'IK_1', 'BK', 'GK', 'NGK', 'PS', 'PZ']
4734
['DEPT', 'GK_1', 'GK', 'IK', 'NGK', 'PS_1', 'PS', 'PZ', 'REZ_1', 'REZ', 'U1', 'U2', 'U3', 'DS', 'BK', 'MPZ', 'GGKP', 'DT', 'MGZ', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZK']
4735
['DEPT', 'BK', 'DS', 'GGKP', 'GK', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZK', 'IK', 'MGZ', 'MPZ', 'NGK', 'PS', 'PZ', 'REZ', 'U1', 'U2', 'U3']
4736
['DEPT', 'BK', 'DS', 'DT', 'GGKP', 'GK', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZK', 'IK', 'MGZ', 'MPZ', 'NGK', 'PS', 'PZ', 'REZ']
4737
['DEPT', 'BK', 'DS', 'GK', 'IK', 'NGK'

In [19]:
#try:
merger.mergeAllCurves()
merger.mergeNoIntersection()
#except Exception as e:
#    print(e)

Merge start
16400
Curve BK started
Curve done
16493
16496
Curve GZ5 started
Curve done
Curve REZ started
Curve done
Curve U3 started
Curve done
4733
Curve IK started
Curve done
4734
Curve GK started
Curve done
Curve PS started
Curve done
Curve REZ started
Curve done
4735
4736
4737
4738
4739
4740
4741
Merge done
Union start
16400
Curve BK started
Curve done
16493
16496
Curve GZ5 started
Curve done
Curve REZ started
Curve done
Curve U3 started
Curve done
4733
Curve IK started
Curve done
4734
Curve GK started
Curve done
Curve PS started
Curve done
Curve REZ started
Curve done
4735
4736
4737
4738
4739
4740
4741
Union end


In [20]:
for wellName in wellsToMerge.keys():
    print(wellName)
    print([curve.name for curve in wellsToMerge[wellName].curvesList])


16400
['DEPT', 'BK', 'BK_1', 'DS', 'GK', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZK', 'MGZ', 'MPZ', 'NGK', 'REZ', 'PS', 'PZ', 'IK']
16493
['DEPT', 'BK', 'DS', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZK', 'MGZ', 'MPZ', 'PS', 'U1', 'U2', 'U3', 'GK', 'NGK', 'PZ', 'IK', 'REZ']
16496
['DEPT', 'BK', 'DS', 'GZ1', 'GZ5', 'GZ5_2', 'GZ5_1', 'IK', 'MGZ', 'MPZ', 'REZ', 'REZ_1', 'U3_1', 'GK', 'NGK', 'PS', 'PZ', 'U1', 'U2', 'U3', 'GZ2', 'GZ3', 'GZ4', 'GZK']
4733
['DEPT', 'DS', 'IK', 'IK_1', 'BK', 'GK', 'NGK', 'PS', 'PZ']
4734
['DEPT', 'GK_1', 'GK', 'IK', 'NGK', 'PS_1', 'PS', 'PZ', 'REZ_1', 'REZ', 'U1', 'U2', 'U3', 'DS', 'BK', 'MPZ', 'GGKP', 'DT', 'MGZ', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZK']
4735
['DEPT', 'BK', 'DS', 'GGKP', 'GK', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZK', 'IK', 'MGZ', 'MPZ', 'NGK', 'PS', 'PZ', 'REZ', 'U1', 'U2', 'U3']
4736
['DEPT', 'BK', 'DS', 'DT', 'GGKP', 'GK', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'GZK', 'IK', 'MGZ', 'MPZ', 'NGK', 'PS', 'PZ', 'REZ']
4737
['DEPT', 'BK', 'DS', 'GK', 'IK', 'NGK'

In [22]:
for wellName in tqdm(wellsToMerge.keys()):
    lasFile = lasio.LASFile()
    lasFile.well['WELL'].value = wellName
    lasFile.well['NULL'].value = -999.25
    lasFile.add_curve('DEPT', data=wellsToMerge[wellName].curves['DEPT'])
    for curve in wellsToMerge[wellName].curvesList:
        if curve.name != 'DEPT':
            lasFile.add_curve(curve.name, data=curve.dataX)
    with codecs.open(converter.replaceSlash('lasFiles/' + wellName + '.las'), 'w', encoding='utf_8') as f:
            lasFile.write(f, version=2, STEP=np.round(lasFile.index[1] - lasFile.index[0], 2))


100%|██████████| 12/12 [00:10<00:00,  1.27it/s]
